# 🏃 Strava Data Streaming Simulator

This notebook simulates real-time activity data streaming into Snowflake, allowing you to test Dynamic Tables refresh behavior.

## What This Does:
- Creates the ACTIVITIES source table
- Generates realistic Strava-like activity data
- Inserts 10 new activities per minute
- Runs continuously to simulate streaming data
- Lightweight and runs natively in Snowflake Notebooks

## Prerequisites:
**Run `00_setup_environment.sql` first** to create database, schemas, role, and warehouses.

## Use Case:
Watch your Dynamic Tables auto-refresh as new data arrives!


## Setup: Import Libraries and Set Context


In [ ]:
# Import required libraries (available in Snowflake Python 3.11)
import time
import random
from datetime import datetime, timedelta
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col

# Get the active Snowflake session
session = get_active_session()

print("✅ Libraries imported successfully!")
print(f"📊 Connected to Snowflake: {session.get_current_database()}.{session.get_current_schema()}")


## Environment Setup
Set the context for data generation


In [ ]:
USE ROLE STRAVA_DEMO_ADMIN;
USE WAREHOUSE STRAVA_DEMO_WH;
USE DATABASE STRAVA_DEMO_SAMPLE;
USE SCHEMA RAW_DATA;


## Create Activities Table
Create the source table if it doesn't exist


In [ ]:
CREATE TABLE IF NOT EXISTS ACTIVITIES (
    activity_id INTEGER,
    athlete_id INTEGER,
    activity_type VARCHAR(50),
    start_date_local TIMESTAMP_NTZ,
    distance_meters FLOAT,
    moving_time_sec INTEGER,
    elapsed_time_sec INTEGER,
    total_elevation_gain_meters FLOAT,
    map_polyline VARCHAR(500),
    average_heartrate INTEGER
);

SELECT 'Activities table ready!' as status;


## Data Generator Function
Creates realistic activity data


In [ ]:
def generate_activity_batch(batch_size=10, starting_id=1000):
    """Generate a batch of realistic Strava activity data"""
    
    activity_types = ['Run', 'Ride', 'Swim', 'Walk', 'Hike']
    activities = []
    
    for i in range(batch_size):
        activity_type = random.choice(activity_types)
        
        # Generate realistic metrics based on activity type
        if activity_type == 'Run':
            distance = random.uniform(3000, 15000)  # 3-15km
            pace_min_per_km = random.uniform(4, 7)
            moving_time = int((distance / 1000) * pace_min_per_km * 60)
            heartrate = random.randint(140, 180)
            elevation = random.uniform(20, 200)
            
        elif activity_type == 'Ride':
            distance = random.uniform(15000, 60000)  # 15-60km
            speed_kmh = random.uniform(20, 30)
            moving_time = int((distance / 1000) / speed_kmh * 3600)
            heartrate = random.randint(120, 160)
            elevation = random.uniform(100, 800)
            
        elif activity_type == 'Swim':
            distance = random.uniform(1000, 3000)  # 1-3km
            pace_min_per_100m = random.uniform(1.5, 2.5)
            moving_time = int((distance / 100) * pace_min_per_100m * 60)
            heartrate = random.randint(130, 170)
            elevation = 0.0
            
        else:  # Walk or Hike
            distance = random.uniform(2000, 10000)  # 2-10km
            pace_min_per_km = random.uniform(10, 15)
            moving_time = int((distance / 1000) * pace_min_per_km * 60)
            heartrate = random.randint(90, 130)
            elevation = random.uniform(50, 400)
        
        elapsed_time = int(moving_time * random.uniform(1.0, 1.3))
        
        # Generate a simple polyline identifier
        athlete_id = random.randint(1, 100)
        map_polyline = f"polyline_{athlete_id}_{starting_id + i}"
        
        activity = {
            'ACTIVITY_ID': starting_id + i,
            'ATHLETE_ID': athlete_id,
            'ACTIVITY_TYPE': activity_type,
            'START_DATE_LOCAL': datetime.now() - timedelta(minutes=random.randint(0, 60)),
            'DISTANCE_METERS': round(distance, 2),
            'MOVING_TIME_SEC': moving_time,
            'ELAPSED_TIME_SEC': elapsed_time,
            'TOTAL_ELEVATION_GAIN_METERS': round(elevation, 2),
            'MAP_POLYLINE': map_polyline,
            'AVERAGE_HEARTRATE': heartrate
        }
        
        activities.append(activity)
    
    return activities

# Test the generator
test_batch = generate_activity_batch(3, starting_id=1)
print("✅ Data generator ready!")
print(f"📝 Sample: {test_batch[0]['ACTIVITY_TYPE']} - {test_batch[0]['DISTANCE_METERS']/1000:.1f}km")


## Stream Data Function
Continuously insert data to simulate streaming


In [ ]:
def stream_activities(duration_minutes=5, batch_size=10, interval_seconds=60):
    """Stream activity data into Snowflake"""
    
    print(f"🚀 Starting data stream...")
    print(f"📊 Config: {batch_size} activities every {interval_seconds} seconds for {duration_minutes} minutes\n")
    
    # Get current max activity_id
    max_id_result = session.sql("SELECT COALESCE(MAX(activity_id), 0) as max_id FROM ACTIVITIES").collect()
    current_id = max_id_result[0]['MAX_ID'] + 1
    
    start_time = time.time()
    end_time = start_time + (duration_minutes * 60)
    batch_count = 0
    total_rows = 0
    
    while time.time() < end_time:
        # Generate and insert batch
        activities = generate_activity_batch(batch_size, starting_id=current_id)
        df = session.create_dataframe(activities)
        df.write.mode("append").save_as_table("ACTIVITIES")
        
        batch_count += 1
        total_rows += batch_size
        current_id += batch_size
        
        elapsed = int(time.time() - start_time)
        remaining = int(end_time - time.time())
        print(f"✅ Batch {batch_count}: {batch_size} activities inserted | Total: {total_rows} | Elapsed: {elapsed}s | Remaining: {remaining}s")
        
        if time.time() < end_time:
            time.sleep(interval_seconds)
    
    print(f"\n🎉 Stream complete! {batch_count} batches, {total_rows} total activities inserted")
    
    final_count = session.sql("SELECT COUNT(*) as total FROM ACTIVITIES").collect()
    print(f"🗂️  Total in table: {final_count[0]['TOTAL']}")

print("✅ Stream function ready!")


## ▶️ Run the Data Stream

Execute this cell to start streaming data.

**Default Configuration:**
- Duration: 5 minutes
- Batch size: 10 activities
- Interval: 60 seconds

*Adjust parameters as needed!*


In [ ]:
# Start streaming - modify parameters as needed
stream_activities(
    duration_minutes=5,
    batch_size=10,
    interval_seconds=60
)


## Verify Data
Check the activities that were just inserted


In [ ]:
-- Summary of recent activities
SELECT 
    activity_type,
    COUNT(*) as count,
    AVG(distance_meters/1000.0) as avg_distance_km,
    AVG(moving_time_sec/60.0) as avg_duration_min
FROM ACTIVITIES
WHERE start_date_local >= DATEADD('hour', -1, CURRENT_TIMESTAMP())
GROUP BY activity_type
ORDER BY count DESC;


In [ ]:
-- Latest 10 activities
SELECT *
FROM ACTIVITIES
ORDER BY start_date_local DESC
LIMIT 10;


In [ ]:
-- Check Dynamic Table refresh status
SELECT 
    name,
    state,
    refresh_start_time,
    refresh_end_time,
    DATEDIFF('second', refresh_start_time, refresh_end_time) as duration_sec
FROM TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY())
WHERE name IN ('ACTIVITY_INTELLIGENCE', 'ATHLETE_PERFORMANCE_DASHBOARD')
ORDER BY refresh_start_time DESC
LIMIT 10;


## 🎉 Complete!

This notebook creates the source table and simulates real-time data streaming for testing Dynamic Tables.

### What You Get:
✅ ACTIVITIES table created in RAW_DATA schema  
✅ Realistic activity data (Run, Ride, Swim, Walk, Hike)  
✅ Configurable streaming (10 activities/minute by default)  
✅ Native Snowflake execution (Python 3.11)  

### Next Steps:
1. Run **02_create_dynamic_tables.ipynb** to create Dynamic Tables
2. Come back to this notebook and run the stream again
3. Watch Dynamic Tables auto-refresh!
4. Use **03_monitoring_queries.sql** to monitor refresh activity
